In [1]:
import os
import subprocess
import logging
import tempfile
import gzip
import pipes

In [2]:
# For chr 1
start = 10177
end = 249240543   # last variant position for chr1
num_vertice_linear_bc = 253384226
num_alt_vertices = 347952
Total_vertices = 253732178

In [3]:
end1 = end - start +1
print(end1)
index = start


249230367


In [4]:
# get the variant position
with open('results/variant_positions_snps_indels_chr1.txt', 'r') as f:
    # removing the new line characters
    variant_positions = [line.rstrip() for line in f]


In [5]:
with open('results/chr1_snps_indel_POS_REF_ALT.txt', 'r') as f:
    # removing the new line characters
    vcf_data_list = [line.rstrip().split() for line in f]

In [6]:
"""
    convert vcf list to vcf dic
"""
# Convert list to dictionary
keys = [vcf_data_list[i][0] for i in range(0,len(vcf_data_list))]
vcf_data_dic ={ k:[] for k in keys }

for i, item in enumerate(vcf_data_list):
    k = keys[i]
    vcf_data_dic[k].append((vcf_data_list[i][1], vcf_data_list[i][2]))


In [9]:
# # Add edges accociated with the linear backbone
# index = start
# with open('chr22_edges.txt', 'w') as f:
#     for i in range(1, end1):
#         cmd ='samtools faidx linear_bc_chr22.fa 22:16050075-51244237:{}-{}| tail -1'.format(i,i)
#         P1 = subprocess.run(cmd, capture_output=True, text=True, shell=True)
#         # label = print(P1.returncode) # will show 0 if execution is successful
#         edge = (index, index+1, P1.stdout.rstrip())
#         # print(edge)
#         index +=1
#         f.write(f"{edge}\n")



In [7]:
# Add edges accociated with the linear backbone
index = start
with open('results/linear_bc_chr1_in_variant_range.fa', 'r') as f:
   for line in f:
      lines = f.read()  # skip the first line
      # lines = f.readlines()[1:]  # skip the first line
      
label_list =[]
for label in lines:
    label_list.append(label)


In [9]:
with open('chr1_edges.txt', 'w') as f:
    for i in range(0, end1):
        label = label_list[i]
        edge = (index, index+1, label)
        # print(edge)
        index +=1
        f.write(f"{edge}\n")

In [ ]:

# Add edges accociated with alternate paths
new_v = num_vertice_linear_bc +1

with open('chr1_edges_2.txt', 'w') as f:
    for pos in variant_positions[0:1700]:
        REF = vcf_data_dic[pos][0][0]
        ALT = vcf_data_dic[pos][0][1]

        ref_len= int(len(str(REF)))
        end_POS_bc= ref_len + int(pos)

        # write outputs to file
        if(int(len(str(ALT))) ==1):
            edge = (pos, end_POS_bc, ALT)
            f.write(f"{edge}\n")

        if(int(len(str(ALT))) !=1):
            for element in ALT:
                if(int(len(str(element))) ==1 and element !=','):
                    edge = (pos, end_POS_bc, element)
                    f.write(f"{edge}\n")
                if(int(len(str(element))) !=1):  
                    v = pos  
                    for i, char in enumerate(element):
                        if(i != len(element)-1):
                            edge = (v, new_v, char)
                            v = new_v
                            new_v +=1
                            f.write(f"{edge}\n")
                        else:
                            edge = (new_v, end_POS_bc, char)
                            f.write(f"{edge}\n")

In [ ]:
# combine two files in a single file
# Combine two files together and keep them sorted


# cat chr${id}_edges.txt chr${id}_edges_2.txt | sort > chr${id}_vg_edges.txt